In [1]:
from haystack.preprocessor.utils import convert_files_to_dicts

12/02/2020 10:42:35 - INFO - faiss -   Loading faiss with AVX2 support.
12/02/2020 10:42:35 - INFO - faiss -   Loading faiss.


In [2]:
dicts=convert_files_to_dicts("/home/elena/Downloads/data/", split_paragraphs=True) # no cleaning function applied

12/02/2020 10:42:46 - INFO - haystack.preprocessor.utils -   Converting /home/elena/Downloads/data/9781839217579-THE_DEEP_LEARNING_WITH_KERAS_WORKSHOP_SECOND_EDITION.pdf


In [3]:
type(dicts), len(dicts), type(dicts[0]), dicts[0].keys()

(list, 1418, dict, dict_keys(['text', 'meta']))

In [4]:
print(dicts[0])

{'text': '\x0cThe Deep Learning\nwith Keras Workshop\nSecond Edition', 'meta': {'name': '9781839217579-THE_DEEP_LEARNING_WITH_KERAS_WORKSHOP_SECOND_EDITION.pdf'}}


In [5]:
from haystack.document_store.faiss import FAISSDocumentStore

document_store = FAISSDocumentStore()
document_store.delete_all_documents()

In [6]:
document_store.write_documents(dicts)

In [7]:
from haystack.retriever.dense import DensePassageRetriever
retriever = DensePassageRetriever(document_store=document_store,
                                  query_embedding_model="facebook/dpr-question_encoder-single-nq-base",
                                  passage_embedding_model="facebook/dpr-ctx_encoder-single-nq-base",
                                  max_seq_len_query=64,
                                  max_seq_len_passage=256,
                                  batch_size=16,
                                  use_gpu=False,
                                  embed_title=True,
                                  use_fast_tokenizers=True)
    

12/02/2020 10:43:56 - WARNING - farm.modeling.language_model -   Could not automatically detect from language model name what language it is. 
	 We guess it's an *ENGLISH* model ... 
	 If not: Init the language model by supplying the 'language' param.
12/02/2020 10:44:05 - WARNING - farm.modeling.language_model -   Could not automatically detect from language model name what language it is. 
	 We guess it's an *ENGLISH* model ... 
	 If not: Init the language model by supplying the 'language' param.


In [8]:
document_store.update_embeddings(retriever)

12/02/2020 10:44:17 - INFO - haystack.document_store.faiss -   Updating embeddings for 1418 docs...
Inferencing Samples: 100%|██████████| 89/89 [11:43<00:00,  7.90s/ Batches]
12/02/2020 10:56:03 - INFO - haystack.document_store.faiss -   Indexing embeddings and updating vectors_ids...
100%|██████████| 1/1 [00:00<00:00,  6.76it/s]


In [9]:
from haystack.reader.farm import FARMReader
farm_reader = FARMReader(model_name_or_path="deepset/roberta-base-squad2", use_gpu=True)

12/02/2020 10:56:28 - INFO - farm.utils -   device: cpu n_gpu: 0, distributed training: False, automatic mixed precision training: None
12/02/2020 10:56:28 - INFO - farm.infer -   Could not find `deepset/roberta-base-squad2` locally. Try to download from model hub ...
Some weights of RobertaModel were not initialized from the model checkpoint at deepset/roberta-base-squad2 and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
12/02/2020 10:56:33 - WARNING - farm.modeling.language_model -   Could not automatically detect from language model name what language it is. 
	 We guess it's an *ENGLISH* model ... 
	 If not: Init the language model by supplying the 'language' param.
12/02/2020 10:56:42 - INFO - farm.utils -   device: cpu n_gpu: 0, distributed training: False, automatic mixed precision training: None
12/02/2020 10:56:42 - INFO - farm.inf

In [10]:
from haystack import Finder

In [11]:
finder = Finder(farm_reader, retriever)

In [12]:
prediction = finder.get_answers(question="What is RNN?", top_k_retriever=10, top_k_reader=5)

Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00,  6.07 Batches/s]
12/02/2020 10:57:08 - INFO - haystack.finder -   Got 10 candidates from retriever
12/02/2020 10:57:08 - INFO - haystack.finder -   Reader is looking for detailed answer in 3458 chars ...
Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00,  1.87 Batches/s]


In [13]:
from haystack.utils import print_answers

In [14]:
print_answers(prediction, details="medium")

[   {   'answer': 'the hidden layer not only gives the\n'
                  'output, but it also feeds back the information of the '
                  'output into itself',
        'context': 'y of the RNN is that the hidden layer not only gives the\n'
                   'output, but it also feeds back the information of the '
                   'output into itself. Before taking a\n'
                   'dee',
        'score': 12.397671699523926},
    {   'answer': 'Recurrent Neural Networks',
        'context': 'Recurrent Neural Networks (RNNs)\n'
                   'RNNs are a class of neural networks that are built on the '
                   'concept of sequential\n'
                   'memory. Unlike traditional neural net',
        'score': 11.025992393493652},
    {   'answer': 'Long Short-Term Memory',
        'context': 'Long Short-Term Memory (LSTM)\n'
                   'LSTMs are RNNs whose main objective is to overcome the '
                   'shortcomings of the vanishin

In [15]:
prediction = finder.get_answers(question="What is a layer?", top_k_retriever=10, top_k_reader=5)

Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00,  7.17 Batches/s]
12/02/2020 10:58:09 - INFO - haystack.finder -   Got 10 candidates from retriever
12/02/2020 10:58:09 - INFO - haystack.finder -   Reader is looking for detailed answer in 5180 chars ...
Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00,  2.42 Batches/s]


In [16]:
print_answers(prediction, details="medium")

[   {   'answer': 'Dense layer',
        'context': 'ras. For now, we will use only the simplest type of\n'
                   'layer, called the Dense layer. A Dense layer is equivalent '
                   'to the fully connected\n'
                   'layers that we h',
        'score': 10.62425708770752},
    {   'answer': 'a\ncomposition of nodes',
        'context': 'ers is part of the Keras core API. A layer can be thought '
                   'of as a\n'
                   'composition of nodes, and at each node, a set of '
                   'computations happen. In Keras, all ',
        'score': 10.560615539550781},
    {   'answer': 'convolutional',
        'context': 'd: it has a four-dimensional input shape (None, 224, 224,\n'
                   '3) and it has three convolutional layers.\n'
                   'The last four layers of the output are as follows:',
        'score': 9.095014572143555},
    {   'answer': 'W1',
        'context': 'st hidden layer. First, the inp